In [1]:
import pandas as pd
import numpy as np
from lr.text_processing.util import pre_process_nli_df
from lr.training.util import label2ternary_label

In [2]:
df = pd.read_csv("data/toy/train.csv")
df.head(5)

,premise,hypothesis,label
0,A couple playing with a little boy on the beach.,A couple are playing with a young child outside.,entailment
1,A man with a red shirt is watching another man...,The men are outside.,neutral
2,A busy city that looks like New York City.,"The city looks like Lawrence, Kansas.",contradiction
3,A blond woman with two children is checking ou...,A mother is with her two children at Walmart b...,neutral
4,Two people pose for the camera.,People pose for a family picture.,neutral


In [3]:
pre_process_nli_df(df)
label2ternary_label(df)
df.head(5)

,premise,hypothesis,label
0,a couple playing with a little boy on the beach,a couple are playing with a young child outside,1
1,a man with a red shirt is watching another man...,the men are outside,0
2,a busy city that looks like new york city,the city looks like lawrence kansas,-1
3,a blond woman with two children is checking ou...,a mother is with her two children at walmart b...,0
4,two people pose for the camera,people pose for a family picture,0
